In [1]:
! pip install -U langchain openai chromadb langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 910.5 kB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 2.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 3.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
! pip install "unstructured[all-docs]==0.10.19" pillow pydantic lxml pillow matplotlib tiktoken open_clip_torch torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 9.7 MB/s eta 0:00:00:00:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.0 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [14]:
from unstructured.partition.pdf import partition_pdf
import os
os.putenv('TESSDATA_PREFIX','/var/lib/dpkg/info/tesseract-ocr-rus.list')
path="/var/mistral_test/"

# Extract images, tables, and chunk text
raw_pdf_elements = partition_pdf(
    filename=path + "Summary_MICEX-RTS_FS_4Q2022_RUS.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    ocr_languages="rus+eng",
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path+"image",
)

# Categorize by type
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [17]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_community.vectorstores import Chroma

# Create chroma w/ multi-modal embeddings
multimodal_embd = Chroma(
    collection_name="multimodal_embd", embedding_function=OpenCLIPEmbeddings()
)

# Get image URIs
image_uris = sorted(
    [
        os.path.join(path+"image/", image_name)
        for image_name in os.listdir(path+"image/")
        if image_name.endswith(".jpg")
    ]
)

# Add images and documents
if image_uris:
    multimodal_embd.add_images(uris=image_uris)
if texts:
    multimodal_embd.add_texts(texts=texts)
if tables:
    multimodal_embd.add_texts(texts=tables)

# Make retriever
retriever_multimodal_embd = multimodal_embd.as_retriever()

In [29]:
import re
from operator import itemgetter
import base64
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
)
from transformers import GenerationConfig

In [30]:
model_name = "/var/mistral_test/Mistral-7B-Instruct-v0.2"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards: 100%|██████████| 3/3 [01:15<00:00, 25.07s/it]


In [33]:
def looks_like_base64(sb):
    """Check if the string looks like base64."""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """Check if the base64 data is an image by looking at the start of the data."""
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def split_image_text_types(docs):
    """Split base64-encoded images and texts."""
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    # Joining the context texts into a single string
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['context']['images'][0]}"
            },
        }
        messages.append(image_message)

    # Adding the text message for analysis
    text_message = {
        "type": "text",
        "text": (
            "Answer the question based only on the provided context, which can include text, tables, and image(s). "
            "If an image is provided, analyze it carefully to help answer the question.\n"
            f"User-provided question / keywords: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """Multi-modal RAG chain"""

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain

In [34]:
chain_multimodal_embd = multi_modal_rag_chain(retriever_multimodal_embd)

In [66]:
import pandas as pd

eval_set = pd.read_excel(path + "вопросы-ответы.xlsx")
eval_set.head(3)

,Вопрос,"Отчёт, из которого взят вопрос",Правильный ответ
0,Группа считает признаками дефолта следующие ви...,"Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 19",контрагент признан несостоятельным (банкротом)...
1,Из чего состоят арендные платежи?,"Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 27",(а) фиксированных платежей (включая по существ...
2,В каких случаях группа переоценивает обязатель...,"Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 27",(а) при изменении срока аренды или изменении о...


In [67]:
import uuid
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Initialize the dataset
dataset_name = f"CPI Eval {str(uuid.uuid4())}"
dataset = []

# Assuming eval_set is a pandas DataFrame
for _, row in eval_set.iterrows():
    # Get Q, A
    q = row["Вопрос"]
    a = row["Правильный ответ"]
    source=row["Отчёт, из которого взят вопрос"]
    # Add the example to the dataset
    dataset.append({"question": q, "answer": a, "source": source})



def generate_response(prompt):
    try:
        inputs = tokenizer(prompt+"формулируй ответы только на русском", return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=150)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response[len(prompt):].strip()
        return response
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Произошла ошибка при генерации ответа."

# Evaluate the model on the dataset
for example in dataset:
    question = example["question"]
    correct_answer = example["answer"]
    source=row["Отчёт, из которого взят вопрос"]
    generated_response = generate_response(question)
    print(f"Question: {question}")
    print(f"source: {source}")
    print(f"Correct Answer: {correct_answer}")
    print(f"Generated Response: {generated_response}")
    print("="*50)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/var/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Группа считает признаками дефолта следующие виды событий
source: Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 9
Correct Answer: контрагент признан несостоятельным (банкротом); 
в суд подан иск третьего лица о признании контрагента несостоятельным (банкротом) и судом вынесено определение о принятии иска к производству; 
контрагент является устойчиво неплатежеспособным, т.е. не выполняет свои обязательства перед Группой в течение срока более 90 календарных дней; 
лицензия контрагента отозвана;
было принято решение о ликвидации в отношении контрагента.
Generated Response: формулируй ответы только на русском языке:

1. Отказ кредитора предоставлять новые кредиты клиенту.
2. Отказ кредитора продлевать срок по текущим кредитам.
3. Отказ кредитора погасить кредиты по сроку.
4. Отказ кредитора погасить кредиты по сниженным условиям.
5. Отказ кредитора погасить кредиты по причине несоответствия кредитоспособности клиента.
6. Отказ кредитора погасить кредиты по при


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Из чего состоят арендные платежи?
source: Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 9
Correct Answer: (а) фиксированных платежей (включая по существу фиксированные платежи) за вычетом любых
стимулирующих платежей;
(б) переменных арендных платежей, зависящих от индекса или ставки, первоначально оцениваемых с
использованием индекса или ставки на дату начала аренды;
(в) сумм, которые, как ожидается, будут уплачены арендатором по гарантиям ликвидационной
стоимости;
(г) цены исполнения опциона на покупку, если имеется достаточная уверенность в том, что арендатор
исполнит этот опцион; и
(д) выплаты штрафов за прекращение аренды, если срок аренды отражает потенциальное исполнение
арендатором опциона на прекращение аренды.
Generated Response: формулируй ответы только на русском языке

Арендные платежи состоят из трёх основных компонентов:

1. Арендная плата (rent): это ежемесячная плата, которую платёк (tenant) платит землепользователю (landlord) за использование недвижимого имущества.
2

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Что такое «LTIP»?
source: Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 9
Correct Answer: LTIP - это программа долгосрочной мотивации, основанная на акциях
Generated Response: формулируй ответы только на русском языке

LTIP (Long-Term Incentive Plan) - это длинносрочная программа вознаграждения, предназначенная для мотивирования и сохранения ключевых сотрудников в компании. Она предусматривает предоставление акций или опционов на их приобретение по благоприятным условиям, которые могут быть реализованы в будущем, в зависимости от достижения определенных целей компанией. ЛТИП помогает сохранить лучших со


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Размер отложенных налоговых обязательств на 31 декабря 2022 года
source: Summary_MICEX-RTS_FS_4Q2022_RUS, стр. 9
Correct Answer: 1464,1 млн. руб.
Generated Response: формулируй ответы только на русском языке

1. По состоянию на 31 декабря 2022 года, размер отложенных налоговых обязательств в Российской Федерации составил 11,3 триллиона рублей.

2. Эта сумма включает в себя налоговые обязательства, которые еще не были погашены по налогам на доходы физических лиц, налогам на доходы юридических лиц, налогу на прибыль юридических лиц, а также налогу на добавленную стоимо


KeyboardInterrupt: 